# Lung Cancer Detection using 3D Swin Transformer - End-to-End Prototype
# LUNA16 Dataset

### This notebook demonstrates the complete pipeline:
### 1. Data Loading & Preprocessing
### 2. Model Training
### 3. Evaluation
### 4. Explainability Visualization

# ============================================================================
# SETUPS AND IMPORTS
# ============================================================================

In [1]:
import os
import sys
import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
sys.path.append('/Users/sreethanubhuvaneshgk/Downloads/desktop/home_folder/lung-cancer-detection/src')

# Import custom modules
from data_loader import LUNA16Dataset, create_data_loaders
from preprocessing import create_preprocessor, create_augmentation
from model import create_model, MultiTaskLoss
from train import Trainer
from evaluate import evaluate_model
from explainability import visualize_predictions_with_explainability

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✓ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

ImportError: cannot import name 'Trainer' from 'train' (/Users/sreethanubhuvaneshgk/Downloads/desktop/home_folder/lung-cancer-detection/src/train.py)

# Load Configuration

In [2]:
# Load config
with open(r'C:\Users\Administrator\Downloads\lung_cancer_detection\configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"  Data directory: {config['data']['root_dir']}")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Number of epochs: {config['training']['num_epochs']}")
print(f"  ROI size: {config['preprocessing']['roi_size']}")
print(f"  Device: {config['hardware']['device']}")

# Set device
device = torch.device(config['hardware']['device'] if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Using device: {device}")

Configuration loaded:


TypeError: 'NoneType' object is not subscriptable

# Data Exploration

In [ ]:
print("\n" + "="*60)
print("DATA EXPLORATION")
print("="*60)

# Load annotations
import pandas as pd
annotations = pd.read_csv(config['data']['annotations_file'])

print(f"\nDataset Statistics:")
print(f"  Total annotations: {len(annotations)}")
print(f"  Unique patients: {annotations['seriesuid'].nunique()}")

# Display sample
print("\nSample annotations:")
print(annotations.head())

# Nodule size distribution
if 'diameter_mm' in annotations.columns:
    print(f"\nNodule diameter statistics:")
    print(annotations['diameter_mm'].describe())
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.hist(annotations['diameter_mm'], bins=30, edgecolor='black')
    plt.xlabel('Diameter (mm)')
    plt.ylabel('Count')
    plt.title('Nodule Size Distribution')
    
    plt.subplot(1, 2, 2)
    if 'malignancy' in annotations.columns:
        malignancy_counts = annotations['malignancy'].value_counts()
        plt.bar(malignancy_counts.index, malignancy_counts.values)
        plt.xlabel('Malignancy')
        plt.ylabel('Count')
        plt.title('Malignancy Distribution')
    
    plt.tight_layout()
    plt.savefig('../results/data_exploration.png', dpi=150)
    plt.show()

# Create Datasets and DataLoaders

In [ ]:
print("\n" + "="*60)
print("CREATING DATASETS")
print("="*60)

# Create augmentation pipeline
augmentation = create_augmentation(config)

# Create datasets
train_dataset = LUNA16Dataset(
    data_dir=config['data']['processed_dir'],
    annotations_file=config['data']['annotations_file'],
    roi_size=tuple(config['preprocessing']['roi_size']),
    transform=augmentation,
    mode='train'
)

val_dataset = LUNA16Dataset(
    data_dir=config['data']['processed_dir'],
    annotations_file=config['data']['annotations_file'],
    roi_size=tuple(config['preprocessing']['roi_size']),
    transform=None,
    mode='val'
)

# Split datasets (80/20 train/val)
from sklearn.model_selection import train_test_split

train_indices, val_indices = train_test_split(
    np.arange(len(train_dataset)),
    test_size=0.2,
    random_state=42
)

train_dataset.samples = [train_dataset.samples[i] for i in train_indices]
val_dataset.samples = [val_dataset.samples[i] for i in val_indices]

print(f"\n✓ Created datasets:")
print(f"  Training samples: {len(train_dataset)}")
print(f"  Validation samples: {len(val_dataset)}")

# Create data loaders
train_loader, val_loader = create_data_loaders(config, train_dataset, val_dataset)

print(f"\n✓ Created data loaders:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")

# Visualize Sample Data

In [ ]:
print("\n" + "="*60)
print("SAMPLE VISUALIZATION")
print("="*60)

# Get a batch
sample_batch = next(iter(train_loader))

print(f"\nBatch shapes:")
print(f"  Images: {sample_batch['image'].shape}")
print(f"  Labels: {sample_batch['label'].shape}")
print(f"  Malignancy: {sample_batch['malignancy'].shape}")
print(f"  BBox: {sample_batch['bbox'].shape}")

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx in range(min(2, sample_batch['image'].shape[0])):
    volume = sample_batch['image'][idx, 0].numpy()
    
    # Show 3 slices
    d = volume.shape[0]
    slices = [d//4, d//2, 3*d//4]
    
    for i, s in enumerate(slices):
        axes[idx, i].imshow(volume[s], cmap='gray')
        axes[idx, i].set_title(f'Sample {idx+1}, Slice {s}')
        axes[idx, i].axis('off')
    
    # Add label info
    label = sample_batch['label'][idx].item()
    mal = sample_batch['malignancy'][idx].item()
    axes[idx, 2].text(
        1.05, 0.5,
        f'Label: {label}\nMalignancy: {mal:.0f}',
        transform=axes[idx, 2].transAxes,
        fontsize=11,
        verticalalignment='center'
    )

plt.tight_layout()
plt.savefig('../results/sample_visualization.png', dpi=150)
plt.show()

# Initialize Model

In [ ]:
print("\n" + "="*60)
print("MODEL INITIALIZATION")
print("="*60)

# Create model
model = create_model(config)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✓ Model created:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size: ~{total_params * 4 / 1024 / 1024:.2f} MB")

# Print model architecture
print("\nModel Architecture:")
print(model)

# Setup Training

In [ ]:
# Create loss function
criterion = MultiTaskLoss(
    detection_weight=1.0,
    malignancy_weight=1.0,
    bbox_weight=0.5
)

# Create trainer
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    config=config,
    device=device
)

print("\n✓ Trainer initialized")

# Train Model

In [ ]:
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

# Train
history = trainer.train()

print("\n✓ Training complete!")

# Plot Training History

In [ ]:
print("\n" + "="*60)
print("TRAINING HISTORY")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss curves
axes[0, 0].plot(history['train_loss'], label='Train Loss')
axes[0, 0].plot(history['val_loss'], label='Val Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Detection loss
det_train = [m['detection_loss'] for m in history['train_metrics']]
det_val = [m['detection_loss'] for m in history['val_metrics']]
axes[0, 1].plot(det_train, label='Train')
axes[0, 1].plot(det_val, label='Val')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Detection Loss')
axes[0, 1].set_title('Detection Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# BBox loss
bbox_train = [m['bbox_loss'] for m in history['train_metrics']]
bbox_val = [m['bbox_loss'] for m in history['val_metrics']]
axes[1, 0].plot(bbox_train, label='Train')
axes[1, 0].plot(bbox_val, label='Val')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('BBox Loss')
axes[1, 0].set_title('Bounding Box Loss')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Malignancy loss
mal_train = [m['malignancy_loss'] for m in history['train_metrics']]
mal_val = [m['malignancy_loss'] for m in history['val_metrics']]
axes[1, 1].plot(mal_train, label='Train')
axes[1, 1].plot(mal_val, label='Val')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Malignancy Loss')
axes[1, 1].set_title('Malignancy Loss')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('../results/training_history.png', dpi=150)
plt.show()


# Load Best Model and Evaluate

In [ ]:

print("\n" + "="*60)
print("EVALUATION")
print("="*60)

# Load best model
best_model_path = os.path.join(config['logging']['checkpoint_dir'], 'best_model.pth')
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])

print(f"✓ Loaded best model from epoch {checkpoint['epoch']}")

# Evaluate
results = evaluate_model(model, val_loader, device)

# Save results
import json
results_path = '../results/evaluation_results.json'

# Convert numpy arrays to lists for JSON serialization
results_serializable = {}
for key, value in results.items():
    if isinstance(value, dict):
        results_serializable[key] = {}
        for k, v in value.items():
            if isinstance(v, np.ndarray):
                results_serializable[key][k] = v.tolist()
            else:
                results_serializable[key][k] = v
    else:
        results_serializable[key] = value

with open(results_path, 'w') as f:
    json.dump(results_serializable, f, indent=2)

print(f"\n✓ Results saved to {results_path}")


# Generate Explainability Visualizations

In [ ]:
print("\n" + "="*60)
print("EXPLAINABILITY VISUALIZATION")
print("="*60)

# Generate Grad-CAM visualizations
visualize_predictions_with_explainability(
    model=model,
    dataloader=val_loader,
    device=device,
    num_samples=5,
    save_dir='../results/explainability/'
)

print("\n✓ Explainability visualizations complete!")


# Summary and Conclusion

In [ ]:
print("\n" + "="*60)
print("PROTOTYPE SUMMARY")
print("="*60)

print(f"\n📊 Final Results:")
print(f"  Detection F1 Score:     {results['detection']['f1_score']:.4f}")
print(f"  Detection AUC:          {results['detection']['auc']:.4f}")
print(f"  Malignancy F1 Score:    {results['malignancy']['f1_score']:.4f}")
print(f"  Malignancy AUC:         {results['malignancy']['auc']:.4f}")
print(f"  Mean IoU:               {results['bbox']['mean_iou']:.4f}")
print(f"  FPs per Scan:           {results['fps_per_scan']:.4f}")

print(f"\n✅ Target Achievement:")
target_met = (
    results['detection']['f1_score'] > 0.80 and
    results['malignancy']['auc'] > 0.85 and
    results['fps_per_scan'] < 2.0
)
print(f"  All Targets Met: {'YES ✓' if target_met else 'NO ✗'}")

print("\n📁 Deliverables Generated:")
print("  ✓ Trained model checkpoint")
print("  ✓ Training history and metrics")
print("  ✓ Evaluation results")
print("  ✓ Grad-CAM explainability visualizations")
print("  ✓ Performance plots")

print("\n" + "="*60)
print("PROTOTYPE COMPLETE!")
print("="*60 + "\n")